In [1]:
!pip install transformers
!pip install wikipedia
!pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import wikipedia as wiki
import nltk
import re
from collections import OrderedDict

In [2]:
wiki.set_lang("pt")
sent_tokenizer = nltk.data.load('tokenizers/punkt/portuguese.pickle')

## Fazendo Download do SQuAD em Português

Trata-se de uma tradução automática para o português pelo Google Tradutor do dataset SQuAD v1.1 em inglês feita pelo grupo Deep Learning Brasil. Vale lembrar que a mesma foi revisada.
https://forum.ailab.unb.br/t/datasets-em-portugues/251/4

Disponível em:
https://drive.google.com/file/d/1Q0IaIlv2h2BC468MwUFmUST0EyN7gNkn

Como a base é muito grande, gerou-se uma nova base de treino (a base de teste não foi reduzida) que corresponde à metade da base inicial. Ela ficará do diretório "squad-pt-small". 
A base original está em "squad-pt". 

## Fazendo o download do script run_squad.py:

O script da huggingface utilizado para fazer o fine tuning do modelo Bert na base SQuAD selecionada.

In [2]:
!curl -L -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/question-answering/run_squad.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 34008  100 34008    0     0  97724      0 --:--:-- --:--:-- --:--:-- 98005


## Fazendo o download do bert pré-treinado PyTorch em português:

Link do projeto da startup brasileira NeuralMind: https://github.com/neuralmind-ai/portuguese-bert

In [1]:
!curl -L -O https://neuralmind-ai.s3.us-east-2.amazonaws.com/nlp/bert-base-portuguese-cased/bert-base-portuguese-cased_pytorch_checkpoint.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0  387M    0 17037    0     0  15334      0  7:21:31  0:00:01  7:21:30 15334
  0  387M    0 1903k    0     0   910k      0  0:07:15  0:00:02  0:07:13  910k
  3  387M    3 12.4M    0     0  4121k      0  0:01:36  0:00:03  0:01:33 4121k
  5  387M    5 21.5M    0     0  5399k      0  0:01:13  0:00:04  0:01:09 5399k
  7  387M    7 27.7M    0     0  5565k      0  0:01:11  0:00:05  0:01:06 5793k
  8  387M    8 32.2M    0     0  5413k      0  0:01:13  0:00:06  0:01:07 6617k
  9  387M    9 36.8M    0     0  5319k      0  0:01:14  0:00:07  0:01:07 7161k
 10  387M   10 41.8M    0     0  5300k      0  0:01:14  0:00:08  0:01:06 6030k
 12  387M   12 47.1M    0     0  5302k      0  0:01

## Fine-tuning do modelo BERT no dataset SQuAD em português

Deve-se rodar os script run_squad.sh (para a base original) e run_squad_small.sh (para a base reduzida).
Optou-se por roda diretamente do terminal, por fora do notebook para que as saídas fossem melhor visualizadas.

In [1]:
#!run_squad.sh
#!run_squad_small.sh

Na máquina local dotada de processador i7-4790k e 16GB de RAM DDR4:

* O run_squad.sh levou 122 horas para rodar e gastou até 15GB de RAM. 
* O run_squad_small.sh levou metade do tempo 61 horas e gastou um pouco mais de 10GB de RAM.

Em relação aos resultados, disponíveis, obteve-se:

In [3]:
Results: {
	'exact': 67.06717123935667, 
	'f1': 79.90285515400765, 
	'total': 10570, 
	'HasAns_exact': 67.06717123935667, 
	'HasAns_f1': 79.90285515400765, 
	'HasAns_total': 10570, 
	'best_exact': 67.06717123935667, 
	'best_exact_thresh': 0.0, 
	'best_f1': 79.90285515400765, 
	'best_f1_thresh': 0.0
}

ResultsSmall: {
	'exact': 64.91012298959319, 
	'f1': 78.2144844407032, 
	'total': 10570, 
	'HasAns_exact': 64.91012298959319, 
	'HasAns_f1': 78.2144844407032, 
	'HasAns_total': 10570, 
	'best_exact': 64.91012298959319, 
	'best_exact_thresh': 0.0, 
	'best_f1': 78.2144844407032, 
	'best_f1_thresh': 0.0
}

O score F1-score é calculado como a média da métrica de similaridade usando F1-score entre cada resposta prevista e cada resposta real utilizando ambas como conjuntos de tokens. 

Observa-se que as métricas foram bem parecidas em ambas as bases. Mesmo com a base reduzida, os resultados foram satisfatórios (com 64.9% de respostas exatas e F1 de 78.2%). Para a base completa, o resultado foi de 67% de respostas exatas e 79.9% de F1-score.

## Carregando o modelo pré-treinado

In [4]:
tokenizer = AutoTokenizer.from_pretrained("./squad-pt-saida/")
model = AutoModelForQuestionAnswering.from_pretrained("./squad-pt-saida/")

## Testando o modelo com um exemplo da wikipedia

Disponível em: https://pt.wikipedia.org/wiki/Brasil

In [5]:
def obtemResposta(model, tokenizer, pergunta, contexto):
    # Gera entrada do modelo:
    inputs = tokenizer.encode_plus(pergunta, contexto, return_tensors="pt") 

    # Obtém os scores:
    resposta_inicio_scores, resposta_fim_scores = model(**inputs)
    resposta_inicio = torch.argmax(resposta_inicio_scores)
    resposta_fim = torch.argmax(resposta_fim_scores) + 1

    # Obtém a melhor resposta:
    return tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][resposta_inicio:resposta_fim]))

In [6]:
contexto = "O território que atualmente forma o Brasil foi oficialmente descoberto pelos portugueses em 22 de abril de 1500, em expedição liderada por Pedro Álvares Cabral. O vínculo colonial foi rompido, de fato, quando em 1808 a capital do reino foi transferida de Lisboa para a cidade do Rio de Janeiro, depois de tropas francesas comandadas por Napoleão Bonaparte invadirem o território português. Em 1815, o Brasil se torna parte de um reino unido com Portugal. Dom Pedro I, o primeiro imperador, proclamou a independência política do país em 1822."

In [7]:
pergunta1 = "Quem proclamou a independência do Brasil?"
print(obtemResposta(model, tokenizer, pergunta1, contexto))

dom pedro i


In [8]:
pergunta2 = "Quem descobriu o Brasil?"
print(obtemResposta(model, tokenizer, pergunta2, contexto))

portugueses


In [9]:
pergunta3 = "Quando o Brasil foi descoberto?"
print(obtemResposta(model, tokenizer, pergunta3, contexto))

22 de abril de 1500


In [10]:
pergunta4 = "Quando o vínculo colonial foi rompido?"
print(obtemResposta(model, tokenizer, pergunta4, contexto))

1808


In [11]:
pergunta5 = "Para onde a capital portuguesa foi transferida?"
print(obtemResposta(model, tokenizer, pergunta5, contexto))

cidade do rio de janeiro


In [12]:
pergunta6 = "Qual era a capital portuguesa antes de ser transferida?"
print(obtemResposta(model, tokenizer, pergunta6, contexto))

lisboa


In [13]:
pergunta7 = "Quem comandou a viagem que encontrou o brasil?"
print(obtemResposta(model, tokenizer, pergunta7, contexto))

pedro alvares cabral


In [14]:
pergunta8 = "Qual o nome da pessoa responsável pelo grupo de pessoas que encontrou o Brasil?"
print(obtemResposta(model, tokenizer, pergunta8, contexto))

pedro alvares cabral


In [15]:
pergunta9 = "O que aconteceu em 1815?"
print(obtemResposta(model, tokenizer, pergunta9, contexto))

brasil se torna parte de um reino unido com portugal


## Criando um modelo de perguntas e respostas com buscas na wikipedia

Baseado no trabalho de Pravesh Bisaria.

Disponível em: https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html

In [16]:
class RespondedorPerguntas:
    def __init__(self, pretrained_model_path='squad-pt-saida'):
        self.tokenizer = AutoTokenizer.from_pretrained(pretrained_model_path)
        self.model = AutoModelForQuestionAnswering.from_pretrained(pretrained_model_path)
        self.max_len = self.model.config.max_position_embeddings
     
    '''
    Realiza tokenizacao retornando uma lista de chunks
    '''
    def __tokenize(self, pergunta, contexto):
        inputs = self.tokenizer.encode_plus(pergunta, contexto, add_special_tokens=True, return_tensors="pt")

        # Caso o input seja grande demais para o modelo (mais de 512 caracteres)
        if len(inputs["input_ids"].tolist()[0]) > self.max_len:
            inputs = self.__chunkify(inputs)
        else:
            inputs = [inputs]

        return inputs

    '''
    Realiza chunkfy
    '''
    def __chunkify(self, inputs):
        # Cria máscara de pergunta (0 é pergunta e 1 é contexto)
        mascara_pergunta = inputs['token_type_ids'].lt(1)
        pergunta = torch.masked_select(inputs['input_ids'], mascara_pergunta)

        # Tamanho do chunk considerando já o token final [SEP]
        tamanho_chunk = self.max_len - pergunta.size()[0] - 1 

        # Cria dict de dict que será povoado com cada chunk de contexto:
        chunked_input = OrderedDict()
        for k,v in inputs.items():
            # Separa pergunta e contexto:
            pergunta = torch.masked_select(v, mascara_pergunta)
            contexto = torch.masked_select(v, ~mascara_pergunta)
            # Faz o split do contexto e itera sobre eles:
            chunks = torch.split(contexto, tamanho_chunk)
            for i, chunk in enumerate(chunks):
                # Concatena pergunta e contexto:
                elem = torch.cat((pergunta, chunk))
                # Apenas se não for o último chunk
                if i != len(chunks)-1:
                    if k == 'input_ids':
                        # Concatena id de token final SEP
                        elem = torch.cat((elem, torch.tensor([102])))
                    else:
                        # Concatena 1 para contexto
                        elem = torch.cat((elem, torch.tensor([1])))

                if i not in chunked_input:
                    chunked_input[i] = {}
                # Gera tensor linha:
                chunked_input[i][k] = torch.unsqueeze(elem, dim=0)
        return list(chunked_input.values())

    '''
    Obtém a resposta de uma pergunta dada um contexto com o score ideal:
    '''
    def __obtem_resposta(self, chunk):
        # Obtém os scores:
        resposta_inicio_scores, resposta_fim_scores = self.model(**chunk)
        resposta_inicio = torch.argmax(resposta_inicio_scores)
        resposta_fim = torch.argmax(resposta_fim_scores) + 1
        resposta = self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(chunk["input_ids"][0][resposta_inicio:resposta_fim]))

        # Obtém a melhor resposta:
        return float(torch.max(resposta_inicio_scores)), float(torch.max(resposta_fim_scores)), resposta

    '''
    Busca páginas da Wikipedia
    TODO: Melhorar busca na wikipedia
    '''
    def __buscaPaginas(self, pergunta, results=5):
        return [wiki.page(p) for p in wiki.search(pergunta, results)]

    '''
    Obtém resposta das N melhores respostas juntamente com as P páginas relacionadas:
    '''
    def obtem_respostas(self, pergunta, numero_paginas=5, top=5, debug=False):
        paginas = self.__buscaPaginas(pergunta, numero_paginas)
        respostas = []
        for ordem_pagina, pagina in enumerate(paginas):
            chunks = self.__tokenize(pergunta, pagina.content)
            if debug:
                print(f"Buscando resposta na página: {pagina.title} - {len(chunks)} trechos de texto")
            for chunk in chunks:
                r = self.__obtem_resposta(chunk)
                if r[2] and r[2] != '[CLS]':
                    # Calcula score pela fórmula (score_inicio + score_fim) * (numero_paginas - ordem_pagina):
                    respostas.append(tuple([pagina.title, (r[0]+r[1])*(numero_paginas-ordem_pagina), r[2]]))
            if debug:
                print(f"Número de respostas identificadas: {len(respostas)}")

        # Obtém as Top N respostas ordenadas:
        return sorted(respostas, key=lambda tup: tup[1], reverse=True)[0:top]

In [17]:
respondedor = RespondedorPerguntas()

In [18]:
pergunta = "Onde Neymar nasceu?"
respondedor.obtem_respostas(pergunta, 5, 5, debug=True)

Buscando resposta na página: Neymar - 43 trechos de texto
Número de respostas identificadas: 4
Buscando resposta na página: Mauricio de Sousa - 8 trechos de texto
Número de respostas identificadas: 5
Buscando resposta na página: Vila Mimosa - 4 trechos de texto
Número de respostas identificadas: 6
Buscando resposta na página: Michel Teló - 13 trechos de texto
Número de respostas identificadas: 8
Buscando resposta na página: Ángel Di María - 9 trechos de texto
Número de respostas identificadas: 8


[('Neymar', 85.02281665802002, 'mogi das cruzes'),
 ('Neymar', 77.76768684387207, 'mogi das cruzes'),
 ('Michel Teló', 34.62398338317871, 'medianeira , no parana'),
 ('Mauricio de Sousa', 20.754725456237793, 'santa isabel , 1935'),
 ('Michel Teló', 20.021625518798828, 'medianeira')]

In [19]:
pergunta = "Como morreu Getúlio Vargas?"
respondedor.obtem_respostas(pergunta, 5, 5, debug=True)

Buscando resposta na página: Getúlio Vargas - 32 trechos de texto
Número de respostas identificadas: 16
Buscando resposta na página: Carta-testamento de Getúlio Vargas - 4 trechos de texto
Número de respostas identificadas: 19
Buscando resposta na página: Estado Novo (Brasil) - 23 trechos de texto
Número de respostas identificadas: 28
Buscando resposta na página: Darci Vargas - 6 trechos de texto
Número de respostas identificadas: 31
Buscando resposta na página: Lutero Vargas - 3 trechos de texto
Número de respostas identificadas: 33


[('Getúlio Vargas', 59.90748882293701, 'suicidio'),
 ('Getúlio Vargas', 43.900707960128784, 'suicidio da mesma forma do pai'),
 ('Getúlio Vargas', 38.3651864528656, 'suicidio com um tiro no coracao'),
 ('Getúlio Vargas', 27.303377985954285, 'suicidio'),
 ('Getúlio Vargas', 22.42054857313633, 'um golpe militar')]

In [20]:
pergunta = "Quando ocorreu a independência do Brasil?"
respondedor.obtem_respostas(pergunta, 5, 5, debug=True)

Buscando resposta na página: Independência do Brasil - 12 trechos de texto
Número de respostas identificadas: 7
Buscando resposta na página: Copa do Brasil de Futebol - 9 trechos de texto
Número de respostas identificadas: 9
Buscando resposta na página: Proclamação da República do Brasil - 11 trechos de texto
Número de respostas identificadas: 11
Buscando resposta na página: Guerras de independência na América espanhola - 10 trechos de texto
Número de respostas identificadas: 13
Buscando resposta na página: Independência ou Morte (Pedro Américo) - 16 trechos de texto
Número de respostas identificadas: 17


[('Independência do Brasil', 79.30469036102295, '7 de setembro de 1822'),
 ('Independência do Brasil', 75.18195152282715, '7 de setembro de 1822'),
 ('Independência do Brasil', 71.680748462677, '7 de setembro de 1822'),
 ('Independência do Brasil', 51.37211799621582, '1820'),
 ('Independência do Brasil', 37.868818044662476, '1821')]

In [21]:
pergunta = "Quanto pesa um cão?"
respondedor.obtem_respostas(pergunta, 5, 5, debug=True)

Buscando resposta na página: American Bully - 5 trechos de texto
Número de respostas identificadas: 2
Buscando resposta na página: Dobermann - 5 trechos de texto
Número de respostas identificadas: 3
Buscando resposta na página: Chihuahua (cão) - 17 trechos de texto
Número de respostas identificadas: 7
Buscando resposta na página: Yorkshire terrier - 18 trechos de texto
Número de respostas identificadas: 11
Buscando resposta na página: Fox terrier - 1 trechos de texto
Número de respostas identificadas: 12


[('Chihuahua (cão)', 46.28483963012695, '1 a 3 kg'),
 ('Yorkshire terrier', 27.234413146972656, 'entre 5 e 7 kg'),
 ('Chihuahua (cão)', 23.80998730659485, '3 kg'),
 ('Yorkshire terrier',
  12.345462799072266,
  '9 , 5 cm no comprimento e 7 , 11 cm'),
 ('Yorkshire terrier', 11.643416166305542, '2 , 3 e os 3 , 5 kg')]

In [22]:
pergunta = "Qual a cor do Sonic?"
respondedor.obtem_respostas(pergunta, 5, 5, debug=True)

Buscando resposta na página: Sonic the Hedgehog - 7 trechos de texto
Número de respostas identificadas: 3
Buscando resposta na página: Lista de personagens de Sonic the Hedgehog - 12 trechos de texto
Número de respostas identificadas: 10
Buscando resposta na página: Lista de personagens de Sonic the Hedgehog - 12 trechos de texto
Número de respostas identificadas: 17
Buscando resposta na página: Sonic - O Filme - 15 trechos de texto
Número de respostas identificadas: 21
Buscando resposta na página: Sonic the Hedgehog - 7 trechos de texto
Número de respostas identificadas: 24


[('Sonic the Hedgehog', 73.69134426116943, 'azul'),
 ('Sonic the Hedgehog', 58.718299865722656, 'vermelho'),
 ('Sonic the Hedgehog', 54.86063003540039, 'verde'),
 ('Lista de personagens de Sonic the Hedgehog', 51.56681251525879, 'roxo'),
 ('Lista de personagens de Sonic the Hedgehog', 44.35158157348633, 'branca')]

In [23]:
pergunta = "Por que a guerra fria foi fria?"
respondedor.obtem_respostas(pergunta, 5, 5, debug=True)

Buscando resposta na página: Guerra Fria - 49 trechos de texto
Número de respostas identificadas: 9
Buscando resposta na página: Espionagem na Guerra Fria - 4 trechos de texto
Número de respostas identificadas: 10
Buscando resposta na página: Guerra Fria - 49 trechos de texto
Número de respostas identificadas: 19
Buscando resposta na página: Guerra Fria - 49 trechos de texto
Número de respostas identificadas: 28
Buscando resposta na página: Guerra Fria (1947–1953) - 12 trechos de texto
Número de respostas identificadas: 31


[('Guerra Fria',
  43.41161012649536,
  'nao houve combates em larga escala diretamente entre as duas superpotencias'),
 ('Guerra Fria',
  40.61384558677673,
  'parecia que qualquer inimigo do regime de bagda era um potencial aliado dos estados unidos'),
 ('Guerra Fria',
  26.046966075897217,
  'nao houve combates em larga escala diretamente entre as duas superpotencias'),
 ('Guerra Fria',
  24.36830735206604,
  'parecia que qualquer inimigo do regime de bagda era um potencial aliado dos estados unidos'),
 ('Guerra Fria',
  21.05194389820099,
  'as tensoes do pos - guerra entre os estados unidos e a uniao sovietica')]